In [ ]:
import tensorflow as tf
import os
import pandas as pd
from google.colab import drive
import sys

# --- KONFIGURASI ---
LEARNING_RATE = 1e-5
JUMLAH_EPOCH = 150
UKURAN_BATCH = 4
UKURAN_GAMBAR = (128, 128, 64, 1)

DRIVE_BASE_PATH = "/content/drive/MyDrive/"
PATH_PETA_FINAL = os.path.join(DRIVE_BASE_PATH, "DATASET/peta_data_final_fix.csv")
PATH_DATASET_OLAHAN = os.path.join(DRIVE_BASE_PATH, "DATASET/")
PATH_HASIL_MODEL = "./hasil_model_cnn_final/"
PATH_LOGS = "./logs_cnn_final/"

if __name__ == "__main__":
    drive.mount('/content/drive')
    if DRIVE_BASE_PATH not in sys.path:
        sys.path.append(DRIVE_BASE_PATH)
    print(f"Path '{DRIVE_BASE_PATH}' telah ditambahkan ke sistem.")

    # Impor komponen dari file final
    from cnn_components_final import build_pure_cnn_model, create_image_only_dataset

    # (Kode cek GPU)
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Ditemukan {len(gpus)} GPU, pelatihan akan dipercepat.")
    else:
        print("Tidak ada GPU, menggunakan CPU.")

    print("Membaca peta data final...")
    df = pd.read_csv(PATH_PETA_FINAL)
    list_latih = df[df['Set'] == 'Latih'][['ID_Pasien', 'Grup']].values.tolist()
    list_validasi = df[df['Set'] == 'Validasi'][['ID_Pasien', 'Grup']].values.tolist()

    print("Mempersiapkan dataset (hanya gambar)...")
    dataset_latih = create_image_only_dataset(list_latih, PATH_DATASET_OLAHAN, UKURAN_BATCH, UKURAN_GAMBAR, augment=True)
    dataset_validasi = create_image_only_dataset(list_validasi, PATH_DATASET_OLAHAN, UKURAN_BATCH, UKURAN_GAMBAR, augment=False)

    print("Membangun model CNN murni...")
    model = build_pure_cnn_model(UKURAN_GAMBAR)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
    )
    model.summary()

    os.makedirs(PATH_HASIL_MODEL, exist_ok=True)
    callbacks_list = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(PATH_HASIL_MODEL, "model_terbaik.h5"),
            monitor='val_recall', mode='max', save_best_only=True, verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=20, verbose=1, restore_best_weights=True
        ),
        tf.keras.callbacks.TensorBoard(log_dir=PATH_LOGS)
    ]

    print("\nMEMULAI PELATIHAN FINAL (MODEL CNN MURNI)...")
    model.fit(
        dataset_latih,
        epochs=JUMLAH_EPOCH,
        validation_data=dataset_validasi,
        callbacks=callbacks_list
    )
    print("\nPELATIHAN FINAL SELESAI!")


Mounted at /content/drive
Path '/content/drive/MyDrive/' telah ditambahkan ke sistem.
Ditemukan 1 GPU, pelatihan akan dipercepat.
Membaca peta data final...
Mempersiapkan dataset (hanya gambar)...
Membangun model CNN murni...


Model: "model_cnn_murni"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_citra_awal (InputLayer)   │ (None, 128, 128, 64,   │             0 │
│                                 │ 1)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 128, 128, 64,   │           224 │
│                                 │ 8)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 64, 64, 32, 8)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32, 8)  │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 64, 64, 32, 16) │         3,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 32, 32, 16, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 16, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 32, 32, 16, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 16, 16, 8, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 8, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     4,194,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_prediksi (Dense)         │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,212,209 (16.07 MB)

 Trainable params: 4,212,097 (16.07 MB)

 Non-trainable params: 112 (448.00 B)


MEMULAI PELATIHAN FINAL (MODEL CNN MURNI)...
Epoch 1/150
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5826 - loss: 0.7784 - precision: 0.3064 - recall: 0.3212
Epoch 1: val_recall improved from -inf to 1.00000, saving model to ./hasil_model_cnn_final/model_terbaik.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.5856 - loss: 0.7753 - precision: 0.3106 - recall: 0.3214 - val_accuracy: 0.7083 - val_loss: 0.4859 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/150
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7411 - loss: 0.6540 - precision: 0.6725 - recall: 0.3411
Epoch 2: val_recall did not improve from 1.00000
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 143ms/step - accuracy: 0.7423 - loss: 0.6504 - precision: 0.6721 - recall: 0.3460 - val_accuracy: 0.9583 - val_loss: 0.3347 - val_precision: 0.8750 - val_recall: 1.0000
Epoch 3/150
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7625 - loss: 0.4571 - precision: 0.5458 - recall: 0.7482
Epoch 3: val_recall did not improve from 1.00000
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 142ms/step - accuracy: 0.7654 - loss: 0.4562 - precision: 0.5535 - recall: 0.7473 - val_accuracy: 1.0000 - val_loss: 0.2017 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/150
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accu

In [ ]:
pip install simpleitk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 50.7 MB/s eta 0:00:00
